In [21]:
import pandas as pd
import json
from pathlib import Path
import yaml

In [27]:
with open('2023_combined_salmon.yaml') as stream:
    try:
        id_classes = yaml.safe_load(stream)['names']
    except yaml.YAMLError as exc:
        print(exc)

In [28]:
class_ids = {}
for key, val in id_classes.items():
    class_ids[val] = key
class_ids

{'Coho': 0,
 'Bull': 1,
 'Rainbow': 2,
 'Sockeye': 3,
 'Pink': 4,
 'Whitefish': 5,
 'Chinook': 6,
 'Shiner': 7,
 'Pikeminnow': 8,
 'Chum': 9,
 'Steelhead': 10,
 'Lamprey': 11,
 'Cutthroat': 12,
 'Stickleback': 13,
 'Sculpin': 14,
 'Jack Coho': 15,
 'Jack Chinook': 16}

In [11]:
with open('project-50-at-2025-01-08-23-48-a71295d5.json') as f:
    vid_annos = json.load(f)

In [12]:
vid_annos

[{'video': 's3://prod-salmonvision-edge-assets-labelstudio-source/HIRMD/tankeeah/jetson-1/motion_vids/HIRMD-tankeeah-jetson-1_20240704_010818_M.mp4',
  'duration': 10.8,
  'Recorded At': 'Jul 04 2024, 01:08:18',
  'counts_left': 0,
  'recorded_at': '2024-07-04T01:08:18.000000Z',
  'counts_right': 0,
  'counts_total': 0,
  'counts_l_Bull': 0,
  'counts_l_Chum': 0,
  'counts_l_Coho': 0,
  'counts_l_Pink': 0,
  'counts_r_Bull': 0,
  'counts_r_Chum': 0,
  'counts_r_Coho': 0,
  'counts_r_Pink': 0,
  'random_number': 0.005096438826186267,
  'counts_l_Shiner': 0,
  'counts_r_Shiner': 0,
  'counts_l_Chinook': 0,
  'counts_l_Lamprey': 0,
  'counts_l_Rainbow': 0,
  'counts_l_Sculpin': 0,
  'counts_l_Sockeye': 0,
  'counts_r_Chinook': 0,
  'counts_r_Lamprey': 0,
  'counts_r_Rainbow': 0,
  'counts_r_Sculpin': 0,
  'counts_r_Sockeye': 0,
  'counts_l_Cutthroat': 0,
  'counts_l_Steelhead': 0,
  'counts_l_Whitefish': 0,
  'counts_r_Cutthroat': 0,
  'counts_r_Steelhead': 0,
  'counts_r_Whitefish': 0,
 

In [19]:
Path(vid_annos[5]['video']).stem

'HIRMD-tankeeah-jetson-1_20240704_052439_M'

In [44]:
# Just make label files for only each keyframe label
# f.write(f"{cls_id} {x} {y} {w} {h} {conf} {unique_id}\n")
KEY_BOX = 'box'
OUTPUT_PATH = Path('tankeeah_salmon_2025-01-08-23-48')
EMPTY_VID_OUTPUT = Path('tankeeah_empty_salmon_2025-01-08-23-48.txt')

OUTPUT_PATH.mkdir(exist_ok=True)
with open(EMPTY_VID_OUTPUT, 'w') as f:
    f.write("")
for vid_anno in vid_annos:
    vid_width = vid_anno['metadata_video_width']
    vid_height = vid_anno['metadata_video_height']
    video = Path(vid_anno['video'])
    vid_dir = OUTPUT_PATH / video.name
    if KEY_BOX in vid_anno:
        try:
            vid_dir.mkdir()
        except FileExistsError:
            print('Anno folder exists:', video)
            continue
        for seq in vid_anno[KEY_BOX]:
            try:
                cls_id = class_ids[seq['labels'][0]]
            except KeyError as e:
                print('Failed class ID:', e)
                continue
            for box in seq['sequence']:
                x = box['x']
                y = box['y']
                w = box['width']
                h = box['height']
                frame_num = box['frame']

                x_center = (x + w / 2) / vid_width
                y_center = (y + h / 2) / vid_height
                w_norm = w / vid_width
                h_norm = h / vid_height

                filename = vid_dir / f"frame_{frame_num:06d}.txt"
                with open(filename, 'a') as f:
                    f.write(f"{cls_id} {x_center} {y_center} {w_norm} {h_norm} 1\n")
    else:
        with open(EMPTY_VID_OUTPUT, 'a') as f:
            f.write(f"{vid_anno['video']}\n")

In [ ]:
ORG = "HIRMD"
SITE = "tankeeah"

download_cmd = ["rclone"]